# For Image Retrieval

In [1]:
### For DataFrame
import pandas as pd
import numpy as np

##### For Query Processing 
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer

### For plotting images
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os import listdir
from PIL import Image as PImage 
import os 
%matplotlib inline

In [2]:
tagged_data_df = pd.read_pickle("tagged_data/tag_data.pkl")
categorical_image_df = pd.read_csv("tagged_data/category_wise_image.csv")

In [3]:
category_df = pd.read_csv("tagged_data/category.csv")

### Query Processing 

In [4]:
def query_processing(sentence):
    # #sentence = input()
    # sentence = 'Fetch the images of persons and handbags'
    sentence = sentence.lower()
    words = nltk.word_tokenize(sentence)

    lemmatizer_words = []
    lemmatizer = WordNetLemmatizer()

    for word in words:
        lemmatizer_words.append(lemmatizer.lemmatize(word, wordnet.VERB))

    ## Stop words Removal
    stop_words = set(stopwords.words("english"))
    refined_words_of_lemmatizer = [word for word in lemmatizer_words if word not in stop_words]
    refined_words_of_lemmatizer = ' '.join(refined_words_of_lemmatizer)

    ## To remove puntuation marks
    final_words_lemmatizer = re.sub(r"[\W]", ' ', refined_words_of_lemmatizer)
    final_words_lemmatizer=final_words_lemmatizer.split()
    
    return final_words_lemmatizer

In [5]:
def stemmer(final_words_lemmatizer):
    stemmer_words= []
    stemmer = PorterStemmer()
    

    for word in final_words_lemmatizer:
        if word.endswith('s'):
            stemmer_words.append(stemmer.stem(word))
        else:
            stemmer_words.append(word)
            
    
    return stemmer_words

In [6]:
def custom_words_removal(object_list):

    words_to_remove = ['image', 'picture',' many', 'fetch', 'get', 'locations', 'location', 'show', 'present']

    object_list = [ i for i in object_list if i not in words_to_remove ]
    

    return object_list

In [7]:
def generate_counter(df, objects_to_detect):

    objects_list = df.objects.values

    counter = list()
    for objects in objects_list:
        c = 0
        for i in objects_to_detect:
            if i in objects:
                c += 1

        counter.append(c)
        
    return counter

In [8]:
def generate_relevance_score(df, not_type):
    
    max_score = max(df.counter)
    
    counter = df.counter.values
    
    if not_type:
        relevance_score = [1 - (i/number_of_objects) for i in counter]
        
    else:
        relevance_score = [i/number_of_objects for i in counter]
    
    return relevance_score

In [9]:
def relevance_score_sorting(df, sentence):
    splitted_sentence = sentence.split()

    max_score = max(tagged_data_df.counter)
    
    
    if 'not' in splitted_sentence:
        if len(df[df.counter == 0].index):
            df['relevance_score'] = generate_relevance_score(df, True)
            ranked_df = df.sort_values('relevance_score', ascending = False)
            return ranked_df

        else:
            return False
            print('Every image has ', obj[0], 'in it.')
    
    else:
        df['relevance_score'] = generate_relevance_score(df, False)
        ranked_df = df.sort_values('relevance_score', ascending = False)
        
        return ranked_df

In [10]:
def check_logical(df, sentence):
    splitted_sentence = sentence.split()

    max_score = max(df.counter)

    if 'and' in splitted_sentence:
        ranked_df = df[df.counter == max_score]
        return ranked_df

    elif 'or' in splitted_sentence:
        ranked_df = df[df.counter != 0]
        return ranked_df

    elif 'not' in splitted_sentence:
        ranked_df = df[df.counter == 0]
        return ranked_df

In [11]:
def object_in_db(df):
    if df['counter'].any():
        return True

    else:
        return False
        print("'",obj[0],"'","can't be found anywhere in the database.")

In [12]:
def show_images(PATH):

    # This is to get the directory that the program 
    # is currently running in. 
    dir_path = os.path.dirname(os.path.realpath(PATH)) 
    dir_path += "\\val2017\\"

    # for root, dirs, files in os.walk(dir_path): 
    #     for file in files: 

    for img in img_names:
        PATH = dir_path + img

        p = PATH
        print(p)
        image = mpimg.imread(p) # images are color images
        plt.gca().clear()
        plt.imshow(image);
        display.display(plt.gcf())

In [13]:
def load_class_names(file_name):
    """Returns a list of class names read from `file_name`."""
    with open(file_name, 'r') as f:
        class_names = f.read().splitlines()
    return class_names

In [14]:
def class_id_to_detect(img_names, object_list, ranked_df, category_df):
    boxes_dicts = list()
    class_id_to_detect = list()

    for img in img_names:
        boxes_dicts.append(ranked_df[ranked_df.image_name == img].repeated_objects_coordinates.values[0][0])


    for objects in object_list:
        if objects in class_names:
            class_id_to_detect.append(category_df[category_df.class_name == objects].index.values[0])

    return [class_id_to_detect, boxes_dicts]

In [15]:
def objects_frequency(obj, ranked_df):

    img_names = []
    repeated_object_len = []

    for i in range(ranked_df.shape[0]):
        for objects in obj:
            if objects in ranked_df.iloc[i][1]:
                if ranked_df.iloc[i][0] not in img_names:
                    img_names.append(ranked_df.iloc[i][0])
                    repeated_object_len.append(len(ranked_df[ranked_df.image_name == ranked_df.iloc[i][0]].repeated_objects.values[0]))
    
    return [img_names, repeated_object_len]


In [16]:
def bounding_box(ranked_df, img_names, boxes_dicts, repeated_object_len, class_id_to_detect, class_names, model_size):
    
#     model_size = ranked_df.iloc[0]['model_size']
    

    for num, img_name, boxes_dict, objects_len in zip(range(len(img_names)), img_names,
                                             boxes_dicts, repeated_object_len):
            img = Image.open('val2017/'+img_name)
            draw = ImageDraw.Draw(img)
            font = ImageFont.truetype(font='files/futur.ttf',
                                      size=(img.size[0] + img.size[1]) // 150)
            resize_factor = \
                (img.size[0] / model_size[0], img.size[1] / model_size[1])

            for cls in class_id_to_detect:
                boxes = boxes_dict[cls]
                if np.size(boxes) != 0:
                    color = np.random.permutation([np.random.randint(256), 255, 0])
                    for box in boxes:

                        xy, confidence = box[:4], box[4]
                        xy = [xy[i] * resize_factor[i % 2] for i in range(4)]
                        x0, y0 = xy[0], xy[1]
                        thickness = (img.size[0] + img.size[1]) // 200
                        for t in np.linspace(0, 1, thickness//4):
                            xy[0], xy[1] = xy[0] + t, xy[1] + t
                            xy[2], xy[3] = xy[2] - t, xy[3] - t
                            draw.rectangle(xy, outline=tuple(color))
                        text = '{}'.format(class_names[cls])

                        text_size = draw.textsize(text, font=font)
                        draw.rectangle([x0, y0 - text_size[1], x0 + text_size[0], y0], fill=tuple(color))
                        draw.text((x0, y0 - text_size[1]), text, fill='black', font=font)



            display(img)

In [17]:
def check_query_type(sentence):
    image_retrieval = False
    is_there_type = False
    location_type = False

    if sentence[:2].lower() == 'is' or sentence[:2].lower() == 'if':
        return 1

    elif sentence.find('where') > 0 or sentence.find('location') > 0:
        return 2

    elif sentence.find('images') > 0 or sentence.find('pictures') > 0 or sentence.find('image') > 0 or sentence.find('picture') > 0:
        return 3

    else:
        return False



In [18]:
def yes_no_type(df, sentence):
    
    ranked_df = relevance_score_sorting(df, sentence)
    
    return ranked_df

In [29]:
sentence = input()

PATH = 'val2017'

class_names = load_class_names('files/coco.names')

model_size = tagged_data_df.iloc[0].model_size

query_result = check_query_type(sentence)


if query_result:
    
    objects_to_detect = stemmer(custom_words_removal(query_processing(sentence)))
    
    number_of_objects = len(objects_to_detect)
    
    tagged_data_df['counter'] = generate_counter(tagged_data_df, objects_to_detect)
    
    if query_result == 1:
        ranked_df = yes_no_type(tagged_data_df, sentence)

        if ranked_df.iloc[0].relevance_score:
            if 'and' in sentence.split() and ranked_df.iloc[0].relevance_score == 1:
                print("Yes in the following image:")
                print(ranked_df.iloc[0]['image_name'][:1])
                
            elif 'not' in sentence.split() and ranked_df.iloc[0].relevance_score == 1:
                print("Yes in the following image:")
                for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                    print(img_name)

            else:
                if ranked_df.iloc[0].relevance_score:
                    print('Sorry, there is no image with all the objects present in it. Only images with either of the objects present in it:')
                    print()
                    for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                        print(img_name)

        else:  
            print("No image could be found.")
                
    if query_result == 2:
        ranked_df = relevance_score_sorting(tagged_data_df, sentence)
        
        if ranked_df.iloc[0].relevance_score:
            if 'and' in sentence.split() and ranked_df.iloc[0].relevance_score == 1:
                print("Yes in the following image:")
                image_names, repeated_object_len = objects_frequency(objects_to_detect, ranked_df[ranked_df.relevance_score == 1])
                class_ids, boxes_dict= class_id_to_detect(image_names, objects_to_detect, ranked_df[ranked_df.relevance_score == 1], category_df)
                bounding_box(ranked_df[ranked_df.relevance_score == 1], image_names, boxes_dict, repeated_object_len, class_ids, class_names, model_size)
                
            else:
                print('Sorry, there is no image with all the objects present in it. Only images with either of the objects present in it:')
                image_names, repeated_object_len = objects_frequency(objects_to_detect, ranked_df[ranked_df.relevance_score >0])
                class_ids, boxes_dict= class_id_to_detect(image_names, objects_to_detect, ranked_df[ranked_df.relevance_score >0], category_df)
                bounding_box(ranked_df[ranked_df.relevance_score > 0], image_names, boxes_dict, repeated_object_len, class_ids, class_names, model_size)
                
        else:  
            print("No image could be found.")
            
            
    if query_result == 3:
        ranked_df = relevance_score_sorting(tagged_data_df, sentence)
        
        if 'not' in sentence.split() and ranked_df.iloc[0].relevance_score == 1:
                print("Yes in the following image:")
                for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                    print(img_name)
        
        elif ranked_df.iloc[0].relevance_score:
            if 'and' in sentence.split() and ranked_df.iloc[0].relevance_score == 1:
                print("Yes in the following image:")
                for img in ranked_df[ranked_df.relevance_score==1]['image_name'].values:
                    print(img)
                

            else:
                print('Sorry, there is no image with all the objects present in it. Only images with either of the objects present in it:')
                print()
                for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                    print(img_name)

        else:  
            print("No image could be found.")

        
    

else:
    print('Sorry!! The query could not be processed. RETRY!!!!')

fetch images with person or handbags in it.
Sorry, there is no image with all the objects present in it. Only images with either of the objects present in it:

handbag.jpg
city.jpg
000000001000.jpg
000000001268.jpg
000000000139.jpg
000000001490.jpg
bike3.jpg
bike2.jpg
bike.jpg
000000002261.jpg
000000002153.jpg
000000002006.jpg
000000001584.jpg
mobile.jpg
000000001353.jpg
000000000785.jpg
000000001296.jpg
000000000872.jpg
000000000885.jpg


In [30]:
ranked_df

,image_name,objects,category,repeated_objects,repeated_objects_coordinates,model_size,counter,relevance_score
31,handbag.jpg,"[person, handbag]","[person, wearable]","[person, person, person, person, handbag, hand...",[{0: [[318.7868 7.9528046 400.81824 409...,NaN,2,1.0
30,city.jpg,"[person, handbag, truck, bus, car, traffic light]","[person, transport, wearable]","[person, person, person, person, person, perso...",[{0: [[340.6137 281.9729 371.4229 ...,NaN,2,1.0
9,000000001000.jpg,"[person, handbag, tennis racket]","[person, sports, wearable]","[person, person, person, person, person, perso...",[{0: [[172.15987 92.2438 230.15309 342...,NaN,2,1.0
10,000000001268.jpg,"[person, handbag]","[person, wearable]","[person, person, person, person, handbag]",[{0: [[260.06006 201.21236 305.25818 288...,NaN,2,1.0
0,000000000139.jpg,"[person, vase, chair, tvmonitor]","[person, gadget, furniture, household]","[person, person, person, chair, chair, tvmonit...",[{0: [[283.42548 164.44724 303.92645 ...,"(416, 416)",1,0.5
14,000000001490.jpg,"[person, surfboard]","[person, sports]","[person, surfboard]",[{0: [[291.6505 159.72043 326.3498 ...,NaN,1,0.5
29,bike3.jpg,"[person, motorbike, bus, car]","[person, transport]","[person, person, person, person, person, perso...",[{0: [[214.96106 84.355705 290.92474 223...,NaN,1,0.5
28,bike2.jpg,"[person, motorbike, car, backpack]","[person, transport, wearable]","[person, person, person, person, person, perso...",[{0: [[191.18695 137.48477 265.04788 ...,NaN,1,0.5
27,bike.jpg,"[person, motorbike]","[person, transport]","[person, motorbike]",[{0: [[215.31717 66.48491 320.7474 327...,NaN,1,0.5
26,000000002261.jpg,"[person, surfboard]","[person, sports]","[person, surfboard]",[{0: [[181.64659 147.3906 249.25099 261...,NaN,1,0.5


In [28]:
show images with cats and person with their locations

SyntaxError: invalid syntax (<ipython-input-28-cf88d1db15dd>, line 1)

In [138]:
objects_to_detect

['person', 'handbag']

In [102]:
image_names

['handbag.jpg',
 'city.jpg',
 '000000001000.jpg',
 '000000001268.jpg',
 '000000000139.jpg',
 '000000001490.jpg',
 'bike3.jpg',
 'bike2.jpg',
 'bike.jpg',
 '000000002261.jpg',
 '000000002153.jpg',
 '000000002006.jpg',
 '000000001584.jpg',
 'mobile.jpg',
 '000000001353.jpg',
 '000000000785.jpg',
 '000000001296.jpg',
 '000000000872.jpg',
 '000000000885.jpg']

In [99]:
class_ids

[0, 26]

In [98]:
ranked_df

,image_name,objects,category,repeated_objects,repeated_objects_coordinates,model_size,counter,relevance_score
31,handbag.jpg,"[person, handbag]","[person, wearable]","[person, person, person, person, handbag, hand...",[{0: [[318.7868 7.9528046 400.81824 409...,NaN,2,1.0
30,city.jpg,"[person, handbag, truck, bus, car, traffic light]","[person, transport, wearable]","[person, person, person, person, person, perso...",[{0: [[340.6137 281.9729 371.4229 ...,NaN,2,1.0
9,000000001000.jpg,"[person, handbag, tennis racket]","[person, sports, wearable]","[person, person, person, person, person, perso...",[{0: [[172.15987 92.2438 230.15309 342...,NaN,2,1.0
10,000000001268.jpg,"[person, handbag]","[person, wearable]","[person, person, person, person, handbag]",[{0: [[260.06006 201.21236 305.25818 288...,NaN,2,1.0
0,000000000139.jpg,"[person, vase, chair, tvmonitor]","[person, gadget, furniture, household]","[person, person, person, chair, chair, tvmonit...",[{0: [[283.42548 164.44724 303.92645 ...,"(416, 416)",1,0.5
14,000000001490.jpg,"[person, surfboard]","[person, sports]","[person, surfboard]",[{0: [[291.6505 159.72043 326.3498 ...,NaN,1,0.5
29,bike3.jpg,"[person, motorbike, bus, car]","[person, transport]","[person, person, person, person, person, perso...",[{0: [[214.96106 84.355705 290.92474 223...,NaN,1,0.5
28,bike2.jpg,"[person, motorbike, car, backpack]","[person, transport, wearable]","[person, person, person, person, person, perso...",[{0: [[191.18695 137.48477 265.04788 ...,NaN,1,0.5
27,bike.jpg,"[person, motorbike]","[person, transport]","[person, motorbike]",[{0: [[215.31717 66.48491 320.7474 327...,NaN,1,0.5
26,000000002261.jpg,"[person, surfboard]","[person, sports]","[person, surfboard]",[{0: [[181.64659 147.3906 249.25099 261...,NaN,1,0.5


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

labelencoder_class_name = LabelEncoder()
labelencoder_category_name = LabelEncoder()

category_df2.class_name = labelencoder_class_name.fit_transform(category_df2.class_name)
category_df2.category_name = labelencoder_category_name.fit_transform(category_df2.category_name)
X = np.array(category_df2.class_name).reshape(-1, 1)

test = pd.DataFrame({'class_name': category_df.class_name, 'class_name_vector': category_df2.class_name, 'category_name': category_df.category_name, 'category_name_vector': category_df2.category_name})
category_df2

In [14]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X, category_df2.category_name)

NameError: name 'DecisionTreeClassifier' is not defined

In [53]:
X_test = np.array(category_df2.class_name[:10]).reshape(-1, 1)
X_test

array([[48],
       [ 9],
       [18],
       [43],
       [ 0],
       [16],
       [73],
       [74],
       [11],
       [72]])

In [54]:
pred = clf.predict(X_test)

In [55]:
pred

array([6, 8, 8, 8, 8, 8, 8, 8, 8, 8])

In [56]:

print(labelencoder_category_name.inverse_transform(pred))

['person' 'transport' 'transport' 'transport' 'transport' 'transport'
 'transport' 'transport' 'transport' 'transport']


In [57]:
for i in range(len(pred)):
    print(pred[i], y_test[i])

NameError: name 'y_test' is not defined

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display